In [1]:
import os
import numpy as np
import logging
from fastapi import FastAPI, HTTPException
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

logging.basicConfig(level=logging.INFO)

app = FastAPI(title="Multi-Document RAG API")

def load_documents(file_paths):
    documents = []
    for file in file_paths:
        try:
            if file.endswith(".pdf"):
                loader = PyPDFLoader(file)
            elif file.endswith(".txt"):
                loader = TextLoader(file)
            else:
                logging.warning(f"Unsupported file format: {file}")
                continue
            documents.extend(loader.load())
        except Exception as e:
            logging.error(f"Error loading {file}: {e}")
    return documents

def process_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(documents)

def create_vector_store(documents):
    embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(documents, embed_model)
#you can upload the document over here 
DATA_DIR = "./data"
file_paths = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith((".pdf", ".txt"))]

documents = load_documents(file_paths)
if not documents:
    raise RuntimeError("No valid documents found!")

chunks = process_documents(documents)
vectorstore = create_vector_store(chunks)

retriever = vectorstore.as_retriever()
llm = Ollama(model="llama3.2")
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

@app.get("/query/")
def query_rag(question: str):
    if not question:
        raise HTTPException(status_code=400, detail="Query cannot be empty.")
    response = qa_chain.run(question)
    return {"response": response}


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data'